In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import sqlite3
import seaborn as sns
from statsbombpy import sb

In [5]:
comps = sb.competitions()
comps = comps[comps['competition_gender'] == 'male']

C:\Users\limwe\anaconda3\envs\sports\lib\site-packages\statsbombpy\api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)


In [ ]:
comp_ids = comps['competition_id'].tolist()
season_ids = comps['season_id'].tolist()
matches = []
for i in range(len(comp_ids)):
    try:
        tmp = sb.matches(competition_id=comp_ids[i],season_id=season_ids[i])
        for idx, row in tmp.iterrows():
            if ('Barcelona' in row['home_team']) or ('Barcelona' in row['away_team']):
                matches.append(row['match_id'])
    except:
        print('No Data Found',comp_ids[i],season_ids[i])

In [ ]:
[['period','timestamp','minute','second','play_pattern','location','team','player','shot_type','shot_body_part','play_pattern','shot_outcome','shot_statsbomb_xg']]

In [ ]:
all_shots = pd.DataFrame()
for match in matches:
    events = sb.events(match_id=match)
    events = events[(events['type']=='Shot') & (events['player']=='Lionel Andrés Messi Cuccittini')]
    all_shots = pd.concat([all_shots,events])


In [160]:
all_shots.to_csv('messi_raw.csv',index=False)

In [159]:
all_shots.dropna(axis=1,how='all').dropna(axis=0,how='all').to_csv('messi.csv')

In [21]:
df = pd.read_csv('Data/messi.csv',index_col=0)


In [ ]:
comp_ids = comps['competition_id'].tolist()
season_ids = comps['season_id'].tolist()
matches = pd.DataFrame()
for i in range(len(comp_ids)):
    try:
        tmp = sb.matches(competition_id=comp_ids[i],season_id=season_ids[i])
        for idx, row in tmp.iterrows():
            if ('Barcelona' in row['home_team']) or ('Barcelona' in row['away_team']):
                test = pd.DataFrame(row[['match_id','match_date','competition','season']]).T
                matches = pd.concat([matches,test])
    except:
        print('No Data Found',comp_ids[i],season_ids[i])

In [22]:
df = df.merge(matches,how='left',on='match_id')

In [23]:
cols = ['location','minute','period','play_pattern','player','second','shot_outcome','shot_body_part','shot_statsbomb_xg','shot_type','competition','season']
goals = df[df['shot_outcome']=='Goal'][cols]
goals['player'] = player

In [24]:
def change_situation(series):
    lst = []
    for x in series:
        if x == 'Other':
            lst.append('Penalty')
        elif x =='From Corner':
            lst.append('Corner')
        elif x ==  'From Free Kick':
            lst.append('Free Kick')
        else:
            lst.append('Open Play')
            
    return lst
goals['play_pattern'] = change_situation(goals['play_pattern'])


In [28]:
goals['season'] = goals['season'].apply(lambda x: x[:4] + '-'+ x[-2:])

In [29]:
goals['x']= goals['location'].apply(lambda x: x.split(',')[0][1:])
goals['y'] = goals['location'].apply(lambda x: x.split(',')[1][:-1])
goals.drop('location',axis=1,inplace=True)

In [30]:
goals.drop(['period','second'],axis=1 ,inplace=True)

In [31]:
goals.drop('shot_type',axis=1,inplace=True)

In [125]:
goals.head()

,minute,play_pattern,player,shot_outcome,shot_body_part,shot_statsbomb_xg,competition,season,x,y
5,53,Penalty,Lionel Messi,Goal,Left Foot,0.023384,Europe - Champions League,2010-11,94.0,43.0
9,69,Penalty,Lionel Messi,Goal,Head,0.328108,Europe - Champions League,2008-09,113.8,36.9
10,12,Open Play,Lionel Messi,Goal,Left Foot,0.041210,Spain - La Liga,2020-21,99.4,33.0
15,89,Free Kick,Lionel Messi,Goal,Left Foot,0.030420,Spain - La Liga,2020-21,94.6,47.5
17,25,Open Play,Lionel Messi,Goal,Left Foot,0.179545,Spain - La Liga,2020-21,106.1,44.8


In [32]:
goals.columns = ['minute','situation','player','result','shotType','xG','league','season','X','Y']

In [33]:
player = 'Lionel Messi'
conn = sqlite3.connect(r"C:\Users\limwe\OneDrive\Documents\Computing Projects\Sports Analytics\Data\soccer_data.db")
query = f"""
SELECT 
minute,result,X,Y,xG,player,shotType,situation,season,league
FROM 
Shots_Data 
WHERE player = "{player}"
AND season = "2021-22"
AND result ='Goal'
"""

psg = pd.read_sql(query,conn)
psg['league'] = 'France - Ligue 1'

In [34]:
psg['shotType'] = 'Left Foot'
psg['situation'] = 'Open Play'

In [35]:
pd.concat([goals,psg]).reset_index(drop=True).to_csv('Data/messi_cleaned.csv',index=False)